In [61]:
#These are the libraries you can use.  You may add any libraries directy related to threading if this is a direction
#you wish to go (this is not from the course, so it's entirely on you if you wish to use threading).  Any
#further libraries you wish to use you must email me, james@uwaterloo.ca, for permission.

from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
import random
from datetime import datetime
from scipy.optimize import minimize

In [63]:
#The following code takes a list of tickers from a csv file, a start date, and an end date. It then generates a dataframe with the daily returns
#of every portfolio within the specified start and end date. It also does this with the two market indexes.

#Another function takes the portfolio as a parameter, and flags every month (in a print function) in which there are 
#less than 18 rows available for any stock. The function then removes that month's data.

#A third function generates a parallel dataframe that shows the percent daily returns of each stock. The percent returns of the two indices are
#calculated and another column showing the final "index" average is made.

In [65]:
START_DATE = '2023-11-25'
END_DATE = '2024-11-19'

In [67]:

ticker_lst = pd.read_csv('Ticker_list.csv')["Ticker"].tolist()
print(ticker_lst)
index_lst = ["XIU.TO","^GSPC"]

exch_rate = yf.download("CADUSD=x",start="2024-11-20", end="2024-11-21")["Close"].iloc[0] #MUST BE CHANGED TO PULL NOVEMBER 22ND

#Takes a yf.Ticker "ticker", checks if the ticker is in canadian dollars. If so, it will just do a simple API pull for the price history.
#If the price is listed in USD, a conversion operation will simply be applied to each price.
def yfin_pull_convert_USD(ticker):
    listed_currency = ticker.fast_info["currency"]
    if listed_currency == "CAD":
        return ticker.history(start=START_DATE, end=END_DATE, interval="1d")["Close"]
    if listed_currency == "USD":
        return ticker.history(start=START_DATE, end=END_DATE, interval="1d")["Close"] * (1/exch_rate)
    else: print("ticker currency is not in USD or CAD error")

#Test cases
#display(yf.Ticker("NVDA").history(start=START_DATE, end=END_DATE, interval="1d")["Close"]
#yfin_pull_convert_USD(yf.Ticker("NVDA"))

#Main function to store the stock prices of the ticker
def gen_tickers(tlist, ilist):
    ret_dataframe = pd.DataFrame()
    for i_str in ilist:
        index_ticker = yf.Ticker(i_str)
        ret_dataframe[i_str] = yfin_pull_convert_USD(index_ticker)
    
    for t_str in tlist:
        ticker = yf.Ticker(t_str)
        ret_dataframe[t_str] = yfin_pull_convert_USD(ticker)
    return ret_dataframe

#Function call: stores a dataframe of index values and stock values
stock_values = gen_tickers(ticker_lst,index_lst)

[*********************100%***********************]  1 of 1 completed

['BNS', 'NVDA', 'JNJ', 'JPM', 'V', 'MFC', 'BCE', 'XOM', 'MSFT', 'HD', 'KO', 'CAT', 'DUK', 'AMT', 'TMO', 'MA', 'SHW', 'NEE', 'TT', 'CEG', 'LVS', 'ABNB', 'GDDY', 'GRMN']


In [68]:
market_label = "Market Returns"
#Function that transforms a list of values into a list of percent returns.
#Also adds a column of the simple average returns of the index tickers.
def convert_pct_returns(stock_value_df):
    ret_dataframe = stock_value_df.interpolate().pct_change()
    ret_dataframe.dropna(inplace=True)
    ret_dataframe[market_label] = ret_dataframe[index_lst].mean(axis=1)
    df_col_order = index_lst + ["Market Returns"] + ticker_lst
    ret_dataframe = ret_dataframe[df_col_order]
    return ret_dataframe

stock_returns = convert_pct_returns(stock_values)

display(stock_returns)

,XIU.TO,^GSPC,Market Returns,BNS,NVDA,JNJ,JPM,V,MFC,BCE,...,TMO,MA,SHW,NEE,TT,CEG,LVS,ABNB,GDDY,GRMN
Date,,,,,,,,,,,,,,,,,,,,,
2023-11-28 00:00:00-05:00,0.000329,0.000980,0.000654,-0.041158,-0.008727,0.002314,0.002285,-0.004722,0.000518,-0.002288,...,-0.008589,0.000098,-0.007008,0.010073,-0.014470,-0.005196,-0.018534,-0.011163,0.003930,-0.000659
2023-11-29 00:00:00-05:00,0.004601,-0.000946,0.001827,0.036321,0.006671,0.003166,0.005080,0.005100,0.000000,-0.003568,...,0.014406,0.001980,0.000987,0.003439,-0.000359,-0.031097,-0.048678,-0.008467,0.008757,-0.003213
2023-11-30 00:00:00-05:00,0.006542,0.003784,0.005163,0.018889,-0.028459,0.016764,0.011405,0.009637,0.015026,0.007417,...,0.005762,0.009785,0.018559,0.002570,0.015909,0.003815,0.017203,-0.001107,0.021855,0.010248
2023-12-01 00:00:00-05:00,0.009750,0.005874,0.007812,0.009828,-0.000107,0.024053,0.004869,-0.000896,0.006636,0.023356,...,0.000746,0.001281,0.012661,0.011280,0.009405,-0.009584,0.014744,0.068704,0.019488,0.011453
2023-12-04 00:00:00-05:00,-0.000644,-0.005409,-0.003026,-0.007520,-0.026836,0.003157,0.007332,-0.007838,0.006085,0.007194,...,-0.001592,-0.014794,-0.003081,-0.008450,0.004175,-0.008008,-0.025000,-0.009776,0.011665,0.004367
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-12 00:00:00-05:00,0.007162,-0.002893,0.002135,-0.001850,0.020859,-0.015480,0.001128,-0.001547,0.003704,-0.005745,...,-0.011281,-0.001792,-0.014335,-0.021736,-0.009110,-0.036119,-0.027251,-0.030568,0.022388,0.004799
2024-11-13 00:00:00-05:00,0.004214,0.000232,0.002223,-0.007228,-0.013622,0.003931,0.006679,-0.001194,-0.002460,-0.017335,...,0.002127,-0.013776,0.009435,-0.000269,0.009388,-0.011981,0.005481,-0.003303,0.008065,-0.002809
2024-11-14 00:00:00-05:00,0.001836,-0.006050,-0.002107,0.004480,0.003350,-0.008940,0.002944,-0.003974,0.013255,-0.013598,...,-0.016387,-0.002836,0.003021,0.014009,-0.011759,-0.002567,-0.020190,0.013558,-0.018279,-0.007559


In [168]:
#Creating the correlation matrix: takes a dataframe of values and a string of columns to drop to make the correlation matrix
def correl(data, dropvalue):
    data_marketdrop = data.drop(labels=dropvalue, axis=1)
    ret_corr = data_marketdrop.corr()
    return ret_corr

stock_correlations = correl(stock_returns, [market_label] + index_lst)
#Apply a format to the DataFrame
display(stock_correlations.style.background_gradient(cmap='RdYlGn_r'))


,BNS,NVDA,JNJ,JPM,V,MFC,BCE,XOM,MSFT,HD,KO,CAT,DUK,AMT,TMO,MA,SHW,NEE,TT,CEG,LVS,ABNB,GDDY,GRMN
BNS,1.000000,0.119392,0.179649,0.247489,0.200575,0.454612,0.205198,0.142701,0.151081,0.396337,0.190067,0.371456,0.237180,0.222782,0.295148,0.229090,0.379537,0.291098,0.267452,0.220267,0.221887,0.223621,0.062366,0.185933
NVDA,0.119392,1.000000,-0.335936,0.068062,0.111584,0.213849,-0.186234,-0.126134,0.473378,0.153123,-0.285839,0.260072,-0.355436,-0.263660,0.078667,0.199246,0.119125,-0.121345,0.327318,0.320030,0.100261,0.364328,0.358200,-0.011532
JNJ,0.179649,-0.335936,1.000000,0.185975,0.180474,0.097488,0.356719,0.160732,-0.097022,0.185603,0.469318,0.029790,0.426702,0.448047,0.340148,0.134393,0.123848,0.249373,-0.218095,-0.219998,0.044647,-0.055399,-0.065546,0.218373
JPM,0.247489,0.068062,0.185975,1.000000,0.345906,0.443363,0.132160,0.308258,0.118748,0.090113,-0.001649,0.490871,0.095893,-0.162295,0.175390,0.312685,0.080765,-0.061774,0.170609,0.087447,0.150808,0.201665,0.236037,0.202008
V,0.200575,0.111584,0.180474,0.345906,1.000000,0.211231,0.028335,0.145261,0.295644,0.156094,0.205266,0.206712,0.176097,0.016258,0.171783,0.755260,0.251751,0.056262,0.181930,0.083613,0.072923,0.118500,0.280684,0.254199
MFC,0.454612,0.213849,0.097488,0.443363,0.211231,1.000000,0.040489,0.278762,0.233827,0.331368,-0.044384,0.480657,0.061897,-0.005643,0.206457,0.261825,0.268134,0.156923,0.275215,0.228478,0.266761,0.326249,0.147051,0.139411
BCE,0.205198,-0.186234,0.356719,0.132160,0.028335,0.040489,1.000000,0.027839,-0.137066,0.213664,0.319767,0.083701,0.391702,0.269506,0.261985,0.044378,0.003094,0.254709,-0.018928,0.062660,0.146319,-0.022565,-0.062476,0.045271
XOM,0.142701,-0.126134,0.160732,0.308258,0.145261,0.278762,0.027839,1.000000,-0.115693,0.127740,0.128015,0.363318,0.145976,0.007109,0.104092,0.145766,0.003863,0.210539,-0.025896,-0.000359,0.204670,0.056393,0.091281,0.037267
MSFT,0.151081,0.473378,-0.097022,0.118748,0.295644,0.233827,-0.137066,-0.115693,1.000000,0.136874,0.035633,0.239963,-0.165714,-0.102614,0.164510,0.329100,0.235923,-0.034879,0.381359,0.189769,0.128516,0.311025,0.319919,0.189467
HD,0.396337,0.153123,0.185603,0.090113,0.156094,0.331368,0.213664,0.127740,0.136874,1.000000,0.185332,0.336112,0.166794,0.374892,0.343323,0.200668,0.515904,0.273260,0.355621,0.153829,0.214048,0.261769,0.079302,0.128619


In [176]:


#finding the ticker coordinates of the lowest correlation value (AI)
def low_correl_ticker_pairs(correlation_matrix):
    if correlation_matrix.shape[0] > 1:
        correl_pairs = correlation_matrix.unstack()
        correl_pairs = correl_pairs[correl_pairs.index.get_level_values(0) != correl_pairs.index.get_level_values(1)]
        lowest_corr_pair = correl_pairs.idxmin()
        lowest_corr_value = correl_pairs.min()
        return list(lowest_corr_pair)
    else:
        remaining_column_ticker = correlation_matrix.columns[0]
        return [remaining_column_ticker]

#recursing through the entire correlation matrix to extract every correlation pair
def corr_pair_extract(correl_matrix):
    #creating a list to store low-correlation pairs into
    low_corr_pairs = []
    while correl_matrix.shape[0] > 0:
        pair = low_correl_ticker_pairs(correl_matrix)
        #removing the tickers after they are packaged into a correlation pair
        if isinstance(pair, str):
            ticker1 = pair
            correl_matrix = correl_matrix.drop(ticker1, axis=0)
            correl_matrix = correl_matrix.drop(ticker1, axis=1)
        else: 
            low_corr_pairs.append(pair)
            correl_matrix = correl_matrix.drop(pair, axis=0)
            correl_matrix = correl_matrix.drop(pair, axis=1)
    return low_corr_pairs

#running the correlation pairing function and storing it in variable "stock pairs"
stock_pairs = corr_pair_extract(stock_correlations)
print(stock_pairs)

[['NVDA', 'DUK'], ['JNJ', 'CEG'], ['AMT', 'GDDY'], ['KO', 'CAT'], ['BCE', 'MSFT'], ['JPM', 'NEE'], ['XOM', 'TT'], ['V', 'LVS'], ['ABNB', 'GRMN'], ['HD', 'MA'], ['MFC', 'TMO'], ['BNS', 'SHW']]


In [210]:
#This part creates lists of stocks, ranging from sizes 12 to 24, 5 of each size, which randomly selects stocks.
def make_port_list(stock_pairs):
    ret_list = []
    #iterates through the minimum selectable stock pairs (6) and the maximum (length of the stock pairs)
    for i in range(6, len(stock_pairs)):
        #simple iteration of 5 times
        for j in range(0,5):
            #generates a random list of indexes to pick from the stock pairs
            rand_list = random.sample(range(0,len(stock_pairs)-1), i)
            portfolio = []
            for k in rand_list:
                portfolio += stock_pairs[k]
            ret_list.append(portfolio)
    return ret_list

port_list = make_port_list(stock_pairs)
display(port_list)
len(port_list)

[['XOM',
  'TT',
  'HD',
  'MA',
  'AMT',
  'GDDY',
  'NVDA',
  'DUK',
  'ABNB',
  'GRMN',
  'V',
  'LVS'],
 ['JNJ',
  'CEG',
  'XOM',
  'TT',
  'AMT',
  'GDDY',
  'ABNB',
  'GRMN',
  'KO',
  'CAT',
  'BCE',
  'MSFT'],
 ['JNJ',
  'CEG',
  'JPM',
  'NEE',
  'V',
  'LVS',
  'NVDA',
  'DUK',
  'BCE',
  'MSFT',
  'KO',
  'CAT'],
 ['MFC',
  'TMO',
  'ABNB',
  'GRMN',
  'AMT',
  'GDDY',
  'KO',
  'CAT',
  'JNJ',
  'CEG',
  'JPM',
  'NEE'],
 ['KO',
  'CAT',
  'V',
  'LVS',
  'XOM',
  'TT',
  'AMT',
  'GDDY',
  'MFC',
  'TMO',
  'JPM',
  'NEE'],
 ['NVDA',
  'DUK',
  'HD',
  'MA',
  'AMT',
  'GDDY',
  'XOM',
  'TT',
  'JNJ',
  'CEG',
  'KO',
  'CAT',
  'JPM',
  'NEE'],
 ['NVDA',
  'DUK',
  'JPM',
  'NEE',
  'JNJ',
  'CEG',
  'BCE',
  'MSFT',
  'AMT',
  'GDDY',
  'MFC',
  'TMO',
  'KO',
  'CAT'],
 ['NVDA',
  'DUK',
  'JPM',
  'NEE',
  'V',
  'LVS',
  'BCE',
  'MSFT',
  'HD',
  'MA',
  'AMT',
  'GDDY',
  'XOM',
  'TT'],
 ['KO',
  'CAT',
  'JNJ',
  'CEG',
  'BCE',
  'MSFT',
  'JPM',
  'NEE',
  'V'

30